clustering final

In [1]:
import sys
sys.path.append('../../../')

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from modules import machine_learning_utils as mlu

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# 1.0 Data retrieval

In [3]:
df = pd.read_pickle("../../../data_lake/output_prod/train.pkl")

# 2.0 Features Engineering

In [4]:
df = mlu.features_eng(df,'clustering')

In [5]:
cols = ['card1', 'card2', 'card3', 'card5', 'M4', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25',
       'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_32', 'id_33', 'id_34',
       'id_35', 'id_36', 'id_37', 'id_38','num_transaction_per_time','addr1','P_emaildomain','R_emaildomain']

In [6]:
df = df.drop(columns=cols)

In [7]:
df = df[df['isFraud']==0]
df = df.drop(columns='isFraud')

In [8]:
df = df.drop(columns=['addr2','multi_transaction_per_time'])

In [9]:
df.sample(1)

,TransactionAmt,ProductCD,card4,card6,DeviceType,num_accounts_related_to_user,num_days_previous_transaction,customer_id,browser_enc,device_info_v4
116123,200.0,R,mastercard,credit,desktop,1440.0,36.583332,10342,chrome,windows


# 3.0 Training clustering

In [10]:
df, imp_mean, imp_mean2, scaler = mlu.clustering_preparation(df,'training')

/anaconda/envs/dev/lib/python3.8/site-packages/sklearn/impute/_base.py:382: FutureWarning:

The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.

/anaconda/envs/dev/lib/python3.8/site-packages/sklearn/impute/_base.py:382: FutureWarning:

The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.

/anaconda/envs/dev/lib/python3.8/site-packages/sklearn/impute/_base.py:382: FutureWarning:

The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.

/anaconda/envs/dev/lib/python3.8/site-packages/sklearn/impute/_base.py:382: FutureWarning:

The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised up

In [11]:
df = mlu.clustering_encoding(df)

# 3.0 Personas generation

In [12]:
mlu.clustering_main(df,'training',5,None)

For cluster number:  2  the score is:  0.26329783
For cluster number:  3  the score is:  0.2508759
For cluster number:  4  the score is:  0.21009842


In [13]:
df, centroid, model = mlu.clustering_main(df,'choosen',None,4)

0    2969
2    2109
3    1918
1    1031
Name: cluster_labels, dtype: int64


In [14]:
centroid.style.background_gradient(cmap='brg',axis=1)

,0,1,2,3
TransactionAmt,0.060090,0.080473,0.027653,0.066001
num_accounts_related_to_user,0.006742,0.005659,0.012320,0.000806
num_days_previous_transaction,0.048834,0.325682,0.059076,0.015510
product_enc,0.000000,3.000000,0.000000,1.000000
card4_enc,0.000000,0.000000,0.000000,0.000000
card6_enc,1.000000,0.000000,0.000000,1.000000
DeviceType_enc,1.000000,1.000000,0.000000,0.000000
browser_enc2,0.000000,0.000000,0.000000,1.000000
device_info_v4_enc,1.000000,1.000000,17.000000,0.000000


# 4.0 Prediction

In [15]:
df = mlu.clustering_prediction(df,model)

In [16]:
df.sample(4)

,TransactionAmt,num_accounts_related_to_user,num_days_previous_transaction,product_enc,card4_enc,card6_enc,DeviceType_enc,browser_enc2,device_info_v4_enc,cluster_labels,cluster_labels_pred
customer_id,,,,,,,,,,,
2028,0.099545,0.000000,0.000342,3,2,1,1,0,1,0,0
1951,0.016170,0.000000,0.000537,1,0,0,1,0,1,0,0
13263,0.053935,0.000000,0.000000,0,0,1,1,0,1,0,0
24281,0.066195,0.000178,0.000415,3,2,1,0,1,0,3,3


# 5.0  Output

In [17]:
with open('../../../data_lake/output_prod/clustering_model_v1.pkl','wb') as file:
    pickle.dump(model, file)

In [18]:
with open('../../../data_lake/output_prod/clustering_imputation_num.pkl','wb') as file:
    pickle.dump(imp_mean, file)

In [19]:
with open('../../../data_lake/output_prod/clustering_imputation_cat.pkl','wb') as file:
    pickle.dump(imp_mean, file)

In [20]:
with open('../../../data_lake/output_prod/clustering_scaler_num.pkl','wb') as file:
    pickle.dump(scaler, file)